In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

def make_graph(stock_data, revenue_data, stock_name):
    """
    Creates a dual-axis plot comparing Historical Share Price and Historical Revenue.

    Args:
        stock_data (pd.DataFrame): DataFrame with stock historical data (must have 'Date' and 'Close').
        revenue_data (pd.DataFrame): DataFrame with revenue data (must have 'Date' and 'Revenue').
        stock_name (str): The name of the stock (e.g., 'Tesla').
    """
    stock_data_specific = stock_data[stock_data.Date <= '2022-04-30']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2022-04-30']

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=("Historical Share Price", "Historical Revenue (Millions of US $)"),
                        vertical_spacing=.3)

    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True),
                             y=stock_data_specific.Close.astype("float"),
                             name="Share Price"), row=1, col=1)

    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True),
                             y=revenue_data_specific.Revenue.astype("float"),
                             name="Revenue"), row=2, col=1)

    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False, height=900, title=f'<b>{stock_name} Stock Price and Revenue Dashboard</b>',
                      xaxis_rangeslider_visible=True)
    fig.show()

In [2]:
# Question 1: Extract Tesla Stock Data using yfinance
print("--- 1. Extracting Tesla (TSLA) Stock Data ---")
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print("Tesla Stock Data Head:")
print(tesla_data.head())
print("-" * 50)

--- 1. Extracting Tesla (TSLA) Stock Data ---
Tesla Stock Data Head:
                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  
--------------------------------------------------


In [3]:
# Question 2: Extract Tesla Revenue Data using web scraping
url_tesla_revenue = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url_tesla_revenue).text
soup = BeautifulSoup(html_data, 'html5lib')

tesla_revenue = pd.read_html(url_tesla_revenue, match="Tesla Quarterly Revenue")[0]
tesla_revenue.columns = ["Date", "Revenue"]

tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(r'[\$,]', '', regex=True)

tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue['Revenue'] = pd.to_numeric(tesla_revenue['Revenue'])

print("Tesla Revenue Data Tail (Cleaned):")
print(tesla_revenue.tail())
print("-" * 50)

Tesla Revenue Data Tail (Cleaned):
          Date  Revenue
48  2010-09-30       31
49  2010-06-30       28
50  2010-03-31       21
52  2009-09-30       46
53  2009-06-30       27
--------------------------------------------------


In [4]:
# Question 3: Extract GameStop Stock Data using yfinance
print("--- 2. Extracting GameStop (GME) Stock Data ---")
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
print("GameStop Stock Data Head:")
print(gme_data.head())
print("-" * 50)

--- 2. Extracting GameStop (GME) Stock Data ---
GameStop Stock Data Head:
                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620129  1.693350  1.603296  1.691667  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716073  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615921  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  
--------------------------------------------------


In [5]:
# Question 4: Extract GameStop Revenue Data using web scraping
url_gme_revenue = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

gme_revenue = pd.read_html(url_gme_revenue, match="GameStop Quarterly Revenue")[0]
gme_revenue.columns = ["Date", "Revenue"]

gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(r'[\$,]', '', regex=True)

gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]
gme_revenue['Revenue'] = pd.to_numeric(gme_revenue['Revenue'])

print("GameStop Revenue Data Tail (Cleaned):")
print(gme_revenue.tail())
print("-" * 50)

GameStop Revenue Data Tail (Cleaned):
          Date  Revenue
57  2006-01-31     1667
58  2005-10-31      534
59  2005-07-31      416
60  2005-04-30      475
61  2005-01-31      709
--------------------------------------------------


In [6]:
# Question 5: Tesla Stock and Revenue Dashboard
print("--- 3. Generating Tesla Dashboard ---")
make_graph(tesla_data, tesla_revenue, 'Tesla')

--- 3. Generating Tesla Dashboard ---


/tmp/ipython-input-1226858658.py:27: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/tmp/ipython-input-1226858658.py:31: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [7]:
# Question 6: GameStop Stock and Revenue Dashboard
print("--- 4. Generating GameStop Dashboard ---")
make_graph(gme_data, gme_revenue, 'GameStop')

--- 4. Generating GameStop Dashboard ---


/tmp/ipython-input-1226858658.py:27: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/tmp/ipython-input-1226858658.py:31: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

